<a href="https://colab.research.google.com/github/julianmadoz/DS_CH_21/blob/main/Desafio_6_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load

In [297]:
import pandas as pd

def load_dataframe_from_url(url, encoding = 'utf-8'):
    path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
    df = pd.read_csv(path, encoding = encoding, sep=';')
    return df

url_w = 'https://drive.google.com/file/d/1cMs7K2IYxHI7LpN1tX7lrjOo0wlEqEdj/view?usp=sharing'
url_p = 'https://drive.google.com/file/d/1suPu2h6TRl-OHMlt54QEjXNsEcA3DFvy/view?usp=sharing'

df_w = load_dataframe_from_url(url_w, encoding = 'ISO-8859-1').set_index('ID')
df_p = load_dataframe_from_url(url_p, encoding = 'ISO-8859-1')

## Clean

In [307]:
print(df_w[df_w.isna().any(axis=1)])
print(df_p[df_p.isna().any(axis=1)])

Empty DataFrame
Columns: [Workclass, fnlwgt, Capital Gain, Capital Loss, Hours per week, Target]
Index: []
Empty DataFrame
Columns: [Age, Sex, ID, Race, Country, Education, Education-Num, Marital Status, Relationship]
Index: []


In [300]:
#safely drop these 2 rows
df_w.dropna(inplace=True)
df_p.dropna(inplace=True)
#change labels
df_w.Target.unique() # returns  [' <=50K', ' >50K', ' <=50K.', ' >50K.']
df_w['Target'] = df_w.Target.apply(lambda x: x[1:] if x[-1]!='.' else x[1:-1]) #get rid of the dot and whitespace. DOT NOT RUN TWICE

## Compare tables

In [301]:
def compare(df1, df2, index):
    df = pd.merge(df1,df2, on=index, how='outer')
    dif = list(df[df.isna().any(axis=1)].ID)
    print('Tienen en común ' + str(df[index].count()) + ' filas en la columna ' + index)
    print('No tienen en comun ' + str(len(dif)) + ' fila(s) en la columna ' + index)
    print('Las filas distintas son: ' + str(dif))
    return df
df = compare(df_w, df_p, 'ID')

Tienen en común 48842 filas en la columna ID
No tienen en comun 1 fila(s) en la columna ID
Las filas distintas son: [1344995623]


## Compare groups

In [302]:
def dist50k(df, column):
    gr = pd.pivot_table(data=df,index='Target',columns=column,values='ID',aggfunc='count')
    gr = 100* gr / gr.sum(axis=0)
    gr = gr.sort_values(by='>50K', axis=1, ascending=False)
    return gr

In [303]:
dist50k(df,'Sex')

Sex,Male,Female
Target,,
<=50K,69.62541,89.074852
>50K,30.37459,10.925148


In [304]:
dist50k(df,'Race')

Race,Asian-Pac-Islander,White,Other,Black,Amer-Indian-Eskimo
Target,,,,,
<=50K,73.074391,74.603099,87.684729,87.91889,88.297872
>50K,26.925609,25.396901,12.315271,12.08111,11.702128


In [305]:
dist50k(df,'Education')

Education,Prof-school,Doctorate,Masters,Bachelors,Assoc-acdm,Assoc-voc,Some-college,HS-grad,12th,7th-8th,10th,9th,5th-6th,11th,1st-4th,Preschool
Target,,,,,,,,,,,,,,,,
<=50K,26.019185,27.441077,45.088446,58.723829,74.203623,74.672489,81.035117,84.142169,92.694064,93.507853,93.736501,94.57672,94.695481,94.922737,96.761134,98.795181
>50K,73.980815,72.558923,54.911554,41.276171,25.796377,25.327511,18.964883,15.857831,7.305936,6.492147,6.263499,5.42328,5.304519,5.077263,3.238866,1.204819
